### 1.构建民法典数据块及分割文档 成为文档块

1. **按条款分割**：将每个法律条款作为一个独立的文本块，因为每个条款通常表达一个完整的法律概念或规定。
2. **保留层次结构**：在元数据中保留编、章、节等层次信息，方便后续检索时进行筛选。
3. **添加上下文**：在每个条款前添加其所属的章节标题，提供必要的上下文信息。
4. **处理长条款**：对于特别长的条款（超过一定字数），可以考虑进一步分割成语义完整的段落。
5. **元数据丰富**：为每个文本块添加丰富的元数据，包括：
   - 编名称（如"物权编"、"合同编"）
   - 章节标题
   - 条款编号
   - 条款类型（定义、权利、义务等）

In [3]:
import re
from typing import List, Dict
import json


def parse_civil_code(text: str) -> List[str]:
    """
    提取法典的层级结构并切分条文，返回条文JSON字符串列表

    返回:
        List[str]: 每个条文转换为JSON字符串的列表
    """
    # 定义正则模式
    part_pattern = re.compile(r"##\s+(.+编)")
    chapter_pattern = re.compile(r"###\s+(.+章)")
    section_pattern = re.compile(r"####\s+(.+节)")
    article_pattern = re.compile(r"\*\*第([零一二三四五六七八九十百千]+)条\*\*\s*(.+)")

    # 初始化当前层级状态
    current_part = ""
    current_chapter = ""
    current_section = ""

    articles = []
    current_article = {"number": "", "content": "", "full_text": ""}

    # 逐行处理
    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue

        # 1. 识别编
        if part_match := part_pattern.match(line):
            current_part = part_match.group(1)
            current_chapter = ""
            current_section = ""

        # 2. 识别章
        elif chapter_match := chapter_pattern.match(line):
            current_chapter = chapter_match.group(1)
            current_section = ""

        # 3. 识别节
        elif section_match := section_pattern.match(line):
            current_section = section_match.group(1)

        # 4. 识别条文
        elif article_match := article_pattern.match(line):
            # 保存上一条文（如果有）
            if current_article["number"]:
                current_article.update({
                    "part": current_part,
                    "chapter": current_chapter,
                    "section": current_section
                })
                articles.append(json.dumps(current_article, ensure_ascii=False))

            # 开始新条文
            article_number = article_match.group(1)
            article_content = article_match.group(2).strip()
            current_article = {
                "number": f"第{article_number}条",
                "content": article_content,
                "full_text": article_content,
                "part": current_part,
                "chapter": current_chapter,
                "section": current_section
            }

        # 5. 条文内容延续
        elif current_article["number"]:
            # 处理内容延续
            current_article["full_text"] += "\n" + line

            # 优化内容显示：连接短行
            if line.endswith(("，", "；", "、")) or len(current_article["content"].split()[-1]) < 4:
                current_article["content"] += line
            else:
                current_article["content"] += " " + line

    # 添加最后一条
    if current_article["number"]:
        articles.append(json.dumps(current_article, ensure_ascii=False))

    return articles

In [4]:
from glob import glob



full_text = ""
file_path = '../rag_resources/milvus_docs/mfd/*.md'
for each_file in glob(file_path,recursive=True):
    with open(each_file,'r') as f:
        full_text = f.read()
text_data = parse_civil_code(full_text)
print(type(text_data[0]))

<class 'str'>


In [5]:
import  os
deepseek_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_url = "https://api.deepseek.com/v1"

In [6]:
# 准备 LLM 和Embedding 模型
from openai import OpenAI
deepseek_client = OpenAI(
    api_key=deepseek_key,
    base_url=deepseek_url
)

In [7]:
# 定义embedding 模型
from pymilvus import model as milvus_model, MilvusClient

embedding_model  = milvus_model.DefaultEmbeddingFunction()


/opt/homebrew/anaconda3/envs/geekbang_aistack_deepseek/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/anaconda3/envs/geekbang_aistack_deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
question_list = [
    "民法典中关于房屋买卖合同的规定有哪些",
    "业主拒缴物业费，物业公司能否以停水停电方式催缴"
]


In [9]:
embedding_query_arr =embedding_model.encode_queries(question_list)
print(len(embedding_query_arr))

2


### 使用 milvus 客户端创建 db

In [11]:
# 创建 milvus collections
from pymilvus import MilvusClient
mdf_milvus_db_path ="./mfd.db"
# 对应表名
collection_name = "mfd_collection"
milvus_client = MilvusClient(
    uri=mdf_milvus_db_path
)
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### 创建 collection

In [13]:
metric_type ="IP"
consistency_level = "Strong"
# 向量维度，至于单个向量维度有关，单个向量维度视模型而定，bert 模型为 768维
dimension_value = len(embedding_query_arr[0])
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=dimension_value,
    metric_type=metric_type,
    consistency_level = consistency_level
)

### 插入数据

In [15]:
from tqdm import  tqdm
# print(text_data)
data =[]
doc_embedding = embedding_model.encode_documents(text_data)
print(f" 原始文本 list 大小为 :{len(text_data)}")
print(f"embedding 成向量后 数组大小为: {len(doc_embedding)}")

 原始文本 list 大小为 :386
embedding 成向量后 数组大小为: 386


In [16]:
### 向 collection 中插入元素
tqdm_desc = f"insert to collection {collection_name} "
for index,line in enumerate(tqdm(text_data,desc=tqdm_desc)):
    item={"id":index,"vector":doc_embedding[index],"text":line}
    data.append(item)
milvus_client.insert(collection_name=collection_name,data=data)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
insert to collection mfd_collection : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 386/386 [00:00<00:00, 254599.99it/s]


{'insert_count': 386, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

### 构建 RAG

In [18]:
query_embedding_vector = embedding_model.encode_queries(question_list)
search_params={"metric_type":"IP","params":{}}
# milvus 返回的字段
output_fields  = ["text"]
# 注意查询是使用 client 来进行 search
search_result = milvus_client.search(
    collection_name=collection_name,
    data= query_embedding_vector,
    limit = 3,
    search_params=search_params,
    output_fields=output_fields
)
print(search_result)


data: [[{'id': 318, 'distance': 0.3773392140865326, 'entity': {'text': '{"number": "第五百二十二条", "content": "当事人可以约定，当一方违约时，向另一方支付一定数额的违约金。 约定的违约金低于造成的损失的，人民法院或者仲裁机构可以根据当事人的请求予以增加；约定的违约金过分高于造成的损失的，人民法院或者仲裁机构可以根据当事人的请求予以适当减少。 #### 第二章 合同的履行", "full_text": "当事人可以约定，当一方违约时，向另一方支付一定数额的违约金。\\n约定的违约金低于造成的损失的，人民法院或者仲裁机构可以根据当事人的请求予以增加；约定的违约金过分高于造成的损失的，人民法院或者仲裁机构可以根据当事人的请求予以适当减少。\\n#### 第二章 合同的履行", "part": "", "chapter": "", "section": ""}'}}, {'id': 194, 'distance': 0.3645155131816864, 'entity': {'text': '{"number": "第三百九十八条", "content": "地役权人有权依照合同约定，利用他人的不动产，以提高自己的不动产的效益。 前款所称他人的不动产为供役地，自己的不动产为需役地。 #### 第四章 担保物权 ##### 第一节 一般规定", "full_text": "地役权人有权依照合同约定，利用他人的不动产，以提高自己的不动产的效益。\\n前款所称他人的不动产为供役地，自己的不动产为需役地。\\n#### 第四章 担保物权\\n##### 第一节 一般规定", "part": "", "chapter": "", "section": ""}'}}, {'id': 299, 'distance': 0.3592635691165924, 'entity': {'text': '{"number": "第五百零三条", "content": "当事人对合同的效力可以约定附条件。 附生效条件的合同，自条件成就时生效。 附解除条件的合同，自条件成就时失效。 当事人为自己的利益不正当地阻止条件成就的，视为条件已经成就；不正当地促成条件成就的，视为条件不成就。", "full_

### 格式化查询 milvus 的查询结果 封装函数获取问题结果

In [20]:
def get_answer_by_query(query_result):
    answer = []
    for each_result in query_result:
        entity_text = json.loads(each_result["entity"]["text"])
        answer.append({
            "id":each_result["id"],
            "distance":each_result["distance"],
            "text":entity_text["full_text"]
        })
    return answer



In [21]:
data = []
for index,answer in enumerate(search_result):
    question = question_list[index]
    answer = get_answer_by_query(search_result[index])
    data.append(answer)

定义系统角色+ 用户 prompt  ，用户提示使用 milvus 提示组装


In [23]:
def _format_answer_to_json(result_dict):
    return json.dumps(result_dict,ensure_ascii=False,indent=4)

In [24]:
context_array  = [ _format_answer_to_json(each_answer_dict) for answer in data for each_answer_dict in answer ]
print(context_array)

['{\n    "id": 318,\n    "distance": 0.3773392140865326,\n    "text": "当事人可以约定，当一方违约时，向另一方支付一定数额的违约金。\\n约定的违约金低于造成的损失的，人民法院或者仲裁机构可以根据当事人的请求予以增加；约定的违约金过分高于造成的损失的，人民法院或者仲裁机构可以根据当事人的请求予以适当减少。\\n#### 第二章 合同的履行"\n}', '{\n    "id": 194,\n    "distance": 0.3645155131816864,\n    "text": "地役权人有权依照合同约定，利用他人的不动产，以提高自己的不动产的效益。\\n前款所称他人的不动产为供役地，自己的不动产为需役地。\\n#### 第四章 担保物权\\n##### 第一节 一般规定"\n}', '{\n    "id": 299,\n    "distance": 0.3592635691165924,\n    "text": "当事人对合同的效力可以约定附条件。\\n附生效条件的合同，自条件成就时生效。\\n附解除条件的合同，自条件成就时失效。\\n当事人为自己的利益不正当地阻止条件成就的，视为条件已经成就；不正当地促成条件成就的，视为条件不成就。"\n}', '{\n    "id": 16,\n    "distance": 0.40003785490989685,\n    "text": "权利人、利害关系人认为不动产登记簿记载的事项错误的，可以申请更正登记。不动产登记簿记载的权利人书面同意或者有证据证明登记确有错误的，登记机构应当予以更正。\\n不动产登记簿记载的权利人不同意更正的，利害关系人可以申请异议登记。登记机构予以异议登记的，申请人在异议登记之日起十五日内不提起诉讼的，异议登记失效。异议登记不当，造成权利人损害的，权利人可以请求损害赔偿。"\n}', '{\n    "id": 23,\n    "distance": 0.3961927592754364,\n    "text": "动产物权设立和转让，合同生效时，所有权没有转移，标的物也未交付，标的物由第三人占有的，出让人将返还请求权转让给受让人时，物权自交付时发生效力。\\n因受让人取得占有的，所有权自交付时发生效力。"

In [47]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
for index, answer in enumerate(data) :
    context_array = [ _format_answer_to_json(each_answer_dict) for each_answer_dict in answer]
    context =  "\n".join(context_array)
    USER_PROMPT = f"""
    请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题,回复格式为：
    用户提问的问题为：xxxx，
    回答：
    ....
    参考依据为:
    ....
    <question>
    {question_list[index]}
    </question>
    
    <context>
    {context}
    </context>

    """
    response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
    print(response.choices[0].message.content)

    用户提问的问题为：民法典中关于房屋买卖合同的规定有哪些，
    回答：
    根据提供的上下文信息，民法典中与房屋买卖合同相关的规定主要包括以下几个方面：
    1. 违约金条款：当事人可以约定违约金，若违约金低于或过分高于实际损失，法院或仲裁机构可应请求予以调整。
    2. 合同效力附条件：合同可约定附生效条件或解除条件，并对不正当干预条件成就的行为作出法律推定。
    3. 地役权相关：涉及不动产利用时，地役权人可依约利用他人不动产以提高自身不动产效益。

    参考依据为:
    - 违约金规定：当事人可约定违约金，法院有权调整不合理违约金（ID:318）
    - 合同附条件：合同效力可附条件，并对条件成就的干预行为有明确规定（ID:299）
    - 地役权条款：不动产利用中的权利义务关系（ID:194）

    （注：由于上下文片段未直接提及"房屋买卖"专项条款，上述回答基于合同通用规则及不动产相关规定的推导）
用户提问的问题为：业主拒缴物业费，物业公司能否以停水停电方式催缴

回答：
根据提供的上下文信息，这些段落主要涉及不动产登记、动产物权转让和预告登记等物权法相关内容，并未提及物业费催缴或停水停电等物业管理的具体规定。因此无法从当前提供的法律条文中得出关于物业公司能否以停水停电方式催缴物业费的明确答案。

参考依据为：
提供的三段上下文均来自《物权法》关于不动产登记（更正登记/异议登记）、动产物权转让效力以及预告登记的规定，与物业管理纠纷无直接关联。建议补充《物业管理条例》或《民法典》合同编相关条款进行针对性解答。
